In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly
import plotly.figure_factory as ff
import scipy.stats
from scipy.integrate import odeint
import pymc3 as pm
import arviz as az
import sunode
import sunode.wrappers.as_theano
# sunode object to customise solver configs
lib = sunode._cvodes.lib
import warnings
warnings.filterwarnings('ignore')

Possible replacement if I prefer to use data from Brazilian cities on the hierarchical modelm

In [2]:
#pd.read_csv("https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-states.csv", date_parser=True).dtypes

In [3]:
class covid_data_etl():
     
    def __init__(self, date_begin='7/11/20', date_end='7/20/20'):
 
        confirmed_cases_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
        self.confirmed_cases = pd.read_csv(confirmed_cases_url, sep=',')
        self.date_begin = date_begin 
        self.date_end = date_end
        self.countries = list()
        self.pops = dict()
        self.cases_obs = dict()
 
    def get_country_data(self, country:str='Brazil', population:float = 212.6e6):
 
        self.countries.append(country)
        self.countries = list(set(self.countries)) # remove duplicates
        self.pops[country] = population
        self.cases_obs[country] = np.array(
            self.confirmed_cases.loc[
                self.confirmed_cases["Country/Region"] == country, self.date_begin:self.date_end
            ]
        )[0]
        print(f"------------ COVID Data for {country}, from {self.date_begin} to {self.date_end}, Loaded ----------- ")

# Continuar do sunode ODE equation

In [4]:
class SIR_model_sunode():
 
    def __init__(self, covid_data_obj) :
        self.covid_data = covid_data_obj

    def _SIR_sunode_rhs_ode(self, t, y, p):
        
        rhs_ode_dict = {
            'S': -p.lam * y.S * y.I,
            'I': p.lam * y.S * y.I - p.mu * y.I,
            #'R': p.f * p.mu * y.I ## f in considered as 1 for this exercice
        }
        
        return rhs_ode_dict
    
    def _setup_SIR_model_data(self, country):
        
        cases_obs = np.array(self.covid_data.cases_obs[country])
        pop_n = self.covid_data.pops[country]
        
        scaled_cases_obs = cases_obs / pop_n
        time_range = np.arange(0,len(cases_obs))
        I_init = scaled_cases_obs[0]
        S_init = 1 - I_init
        self.country_sir_input = {
            'country':country,
            'I_init': I_init,
            'S_init': S_init,
            'time_range': time_range,
            'scaled_cases_obs':scaled_cases_obs
        }
    
    def build_pymc_sir_model(self, country, prior, likelihood):
        
        self._setup_SIR_model_data(country)
 
        with pm.Model() as model:
            ## pymc RVs - Priors
            sigma = pm.HalfCauchy('sigma', likelihood['sigma'], shape=1)
            lam = pm.Lognormal('lambda', prior['lam'], prior['lambda_std'])
            mu = pm.Lognormal('mu', prior['mu'], prior['mu_std'])
            
            ## sunode ODE equation and gradients computed using sunode
            res, _, problem, solver, _, _ = sunode.wrappers.as_theano.solve_ivp(
                y0={'S': (self.country_sir_input['S_init'], ()), 
                    'I': (self.country_sir_input['I_init'], ()),},
                params={'lam': (lam, ()), 'mu': (mu, ()), '_dummy': (np.array(1.), ())},
                rhs=self._SIR_sunode_rhs_ode,
                tvals=self.country_sir_input['time_range'],
                t0=self.country_sir_input['time_range'][0]
            )
            ## raw sundials functions to customise sunode solver options
            ## powered by pysundials https://github.com/jmuhlich/pysundials/tree/master/doc
            lib.CVodeSStolerances(solver._ode, 1e-10, 1e-10)
            lib.CVodeSStolerancesB(solver._ode, solver._odeB, 1e-8, 1e-8)
            lib.CVodeQuadSStolerancesB(solver._ode, solver._odeB, 1e-8, 1e-8)
            lib.CVodeSetMaxNumSteps(solver._ode, 5000)
            lib.CVodeSetMaxNumStepsB(solver._ode, solver._odeB, 5000)
            
            ## pymc RVs - likelihood
            if(likelihood['distribution'] == 'lognormal'):
                I = pm.Lognormal(
                    'I', mu=res['I'], sigma=sigma, 
                    observed=self.country_sir_input['scaled_cases_obs']
                )
            elif(likelihood['distribution'] == 'normal'):
                I = pm.Normal(
                    'I', mu=res['I'], sigma=sigma, 
                    observed=self.country_sir_input['scaled_cases_obs']
                )
            elif(likelihood['distribution'] == 'students-t'):
                I = pm.StudentT( 
                    "I",  
                    nu=likelihood['nu'],       # degrees of freedom
                    mu=res['I'],     # likelihood distribution mean, these are the predictions from SIR
                    sigma=sigma, 
                    observed=self.country_sir_input['scaled_cases_obs']
                )
            
            ## pymc deterministic params
            R0 = pm.Deterministic('R0',lam/mu)
        
        self.pymc_model = model
        
    def _setup_SIR_hier_model_data(self, specific_country_list = None):
        
        if specific_country_list is not None:
            country_list = [
                key for key, value in covid_obj.cases_obs.items() if key in specific_country_list
            ]
        else:
            country_list = [key for key, value in covid_obj.cases_obs.items()]
        
        scaled_cases_obs_list = [
            value / covid_obj.pops[key] 
            for key, value in covid_obj.cases_obs.items() 
            if key in country_list
        ]
        
        scaled_cases_obs_array = np.array(scaled_cases_obs_list).T 
        time_range_hier = np.arange(0,len(scaled_cases_obs_array))
        I_init_array =  scaled_cases_obs_array[0]
        S_init_array = 1 - I_init_array
        
        self.countries_hier_model_input = {
            'country_list':country_list,
            'scaled_cases_obs_array': scaled_cases_obs_array,
            'time_range_hier': time_range_hier,
            'I_init_array': I_init_array,
            'S_init_array':S_init_array
        }
    
    def build_pymc_sir_hier_model(self, hyper_prior, prior, 
                                  likelihood, specific_country_list = None):
        
        self._setup_SIR_hier_model_data(specific_country_list)
        shape = len(self.countries_hier_model_input['country_list'])
        
        with pm.Model() as model:
            ## pymc RVs - Hyper priors
            prior_lam_mu = pm.Lognormal(
                'prior_lam_mu', hyper_prior['prior_lam_mu_mu'], hyper_prior['prior_lam_mu_std']
            ) 
            prior_mu_mu = pm.Lognormal(
                'prior_mu_mu', hyper_prior['prior_mu_mu_mu'], hyper_prior['prior_mu_mu_std']
            ) 
            
            ## pymc RVs - Priors
            lam_std = pm.HalfNormal('lam_std', prior['lam_std'])
            mu_std = pm.HalfNormal('mu_std', prior['mu_std'])
            lam = pm.Lognormal('lambda', prior_lam_mu, lam_std, shape=shape)
            mu = pm.Lognormal('mu', prior_mu_mu, mu_std, shape=shape)
            # checking why it was deterministic
            #sigma = pm.HalfCauchy('sigma', likelihood['sigma'], shape=1) 
            
            ## sunode ODE equation and gradients computed using sunode         
            res, _, problem, solver, _, _ = sunode.wrappers.as_theano.solve_ivp(
                y0={ # We need to specify the shape of each parameter. Any empty tuple corresponds to a scalar value.
                   'S': (self.countries_hier_model_input['S_init_array'], (shape,)), 
                   'I': (self.countries_hier_model_input['I_init_array'], (shape,))
                },
                params={
                    'lam': (lam, (shape,)),
                    'mu': (mu, (shape,)),
                    '_dummy': (np.array(1.), ())
                },
                rhs=self._SIR_sunode_rhs_ode,
                # The time points where we want to access the solution
                tvals=self.countries_hier_model_input['time_range_hier'][1:],
                t0=self.countries_hier_model_input['time_range_hier'][0],
            )
            
            ## raw sundials functions to customise sunode solver options
            ## powered by pysundials https://github.com/jmuhlich/pysundials/tree/master/doc
            lib.CVodeSStolerances(solver._ode, 1e-10, 1e-10)
            lib.CVodeSStolerancesB(solver._ode, solver._odeB, 1e-8, 1e-8)
            lib.CVodeQuadSStolerancesB(solver._ode, solver._odeB, 1e-8, 1e-8)
            lib.CVodeSetMaxNumSteps(solver._ode, 5000)
            lib.CVodeSetMaxNumStepsB(solver._ode, solver._odeB, 5000)
            
            ## pymc RVs - likelihood
            sigma = likelihood['sigma']
            I = pm.Normal(
                'I', mu=res['I'], sigma=sigma, 
                observed=self.countries_hier_model_input['scaled_cases_obs_array'][1:]
            )
            
            ## pymc deterministic params
            R0 = pm.Deterministic('R0',lam/mu)
    
        self.hier_pymc_model = model
        
    #def plot_pymc_model_dag(self):
    #    
    #    dag_fig = pm.model_to_graphviz(self.pymc_model)
    #    return dag_fig
    
    def sample_posterior_pymc_model(
        self, n_samples, hier_sir,
        n_tune, n_chains, n_cores
    ):
        if hier_sir:
            model = self.hier_pymc_model
            self.last_posterior_sampling = 'hier_sir'
        else:
            model = self.pymc_model
            self.last_posterior_sampling = 'standard_sir'
            
        self.n_samples = n_samples
        self.n_tune = n_tune
        self.n_chains = n_chains
        self.n_cores = n_cores
        
        try:
            model is not None
        except NotImplementedError as error:
            print('pymc3 model instance not found')

        with model:
            
            trace = pm.sample(self.n_samples, tune=self.n_tune, 
                              chains=self.n_chains, cores=self.n_cores)
            
        self.pymc_model_trace = trace
        
    
    def pymc_model_posterior_summary(self):
        
        trace_summary = az.summary(self.pymc_model_trace)
        return trace_summary
    
    
    def pymc_model_plot_posterior(self):
        
        data = az.from_pymc3(trace=self.pymc_model_trace)
        az.plot_posterior(data, round_to=2, point_estimate='mode', hdi_prob=0.95)

    def pymc_model_plot_traces(self):
        
        axes = az.plot_trace(self.pymc_model_trace)
        axes.ravel()[0].figure
    
    
    def pymc_model_plot_interactive_trace(self, trace='R0'):
        
        fig = ff.create_distplot([self.pymc_model_trace[trace]], bin_size=0.5, group_labels=['x'])
        # Add title
        fig.update_layout(title_text='Curve and Rug Plot')
        fig.update_xaxes(range=[0,7])
        return fig

In [5]:
# -------- COVID Data --------#
covid_obj = covid_data_etl(date_begin='3/1/20', date_end='9/28/20')
covid_obj.get_country_data(country='Brazil', population= 212.6e6)
covid_obj.get_country_data(country='US', population= 212.6e6) # adjust pop

------------ COVID Data for Brazil, from 3/1/20 to 9/28/20, Loaded ----------- 
------------ COVID Data for US, from 3/1/20 to 9/28/20, Loaded ----------- 


### Single Country Sample

In [8]:
# -------- SIR Params --------#
likelihood = {'distribution': 'lognormal', 
              'sigma': 2}
prior = {'lam': 1.0, 
         'mu': 0.5, 
         'lambda_std': 1.0,
         'mu_std': 0.2 }
# -------- SIR Obj Loading --------#
sir_model = SIR_model_sunode(covid_obj)
sir_model.build_pymc_sir_model(country='Brazil', likelihood=likelihood, prior=prior)
#sir_model.plot_pymc_model_dag() ## understand why the image didn't keep python-graphviz installed

In [9]:
n_samples = 2000
n_tune = 1000
n_chains = 4
n_cores = 4

sir_model.sample_posterior_pymc_model(n_samples=nsamples, n_tune=n_tune, 
                                      n_chains=n_chains, n_cores=n_cores)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [mu, lambda, mu_std, lam_std, prior_mu_mu, prior_lam_mu]



[CVODES ERROR]  CVode
  At t = 4.45615 and h = 6.26372e-07, the error test failed repeatedly or with |h| = hmin.


[CVODES ERROR]  CVode
  At t = 4.45615 and h = 6.26372e-07, the error test failed repeatedly or with |h| = hmin.


[CVODES ERROR]  CVode
  At t = 0 and h = 4.52504e-55, the corrector convergence test failed repeatedly or with |h| = hmin.


[CVODES ERROR]  CVode
  At t = 0 and h = 4.52504e-55, the corrector convergence test failed repeatedly or with |h| = hmin.


[CVODES ERROR]  CVode
  At t = 211, mxstep steps taken before reaching tout.


[CVODEA ERROR]  CVodeB
  Error occured while integrating backward problem # 0


[CVODES ERROR]  CVode
  At t = 210.999, mxstep steps taken before reaching tout.


[CVODEA ERROR]  CVodeB
  Error occured while integrating backward problem # 0


[CVODES ERROR]  CVode
  At t = 210.999 repeated recoverable right-hand side function errors.


[CVODEA ERROR]  CVodeB
  Error occured while integrating backward problem # 0


[CVODES ERROR]  CVode


KeyboardInterrupt: 

In [ ]:
sir_model.pymc_model_posterior_summary()

In [ ]:
sir_model.pymc_model_plot_posterior()

In [ ]:
sir_model.pymc_model_plot_traces()

In [ ]:
sir_model.pymc_model_plot_interactive_trace()

### Hier SIR

In [6]:
hyper_prior= {
    'prior_lam_mu_mu': 0.75,
    'prior_lam_mu_std': 2,
    'prior_mu_mu_mu': 0.75,
    'prior_mu_mu_std': 2
}

prior = {'lam_std': 1.0,'mu_std': 0.2 }

likelihood = {'sigma': 0.01}
# -------- SIR Obj Loading --------#
sir_model = SIR_model_sunode(covid_obj)
sir_model.build_pymc_sir_hier_model(hyper_prior=hyper_prior, prior=prior, likelihood=likelihood)
#sir_model.plot_pymc_model_dag() ## understand why the image didn't keep python-graphviz installed

In [ ]:
n_samples = 2000
n_tune = 1000
n_chains = 4
n_cores = 4

sir_model.sample_posterior_pymc_model(
    n_samples=n_samples, n_tune=n_tune, hier_sir=True,
    n_chains=n_chains, n_cores=n_cores
)